Kita h5pyd is still on 0.3.2, so need to pip install latest version of h5pyd:
```
pip install git+https://github.com/HDFGroup/h5pyd.git
```

In [ ]:
# !pip install git+https://github.com/HDFGroup/h5pyd.git 

In [1]:
%matplotlib inline
import h5pyd as h5py
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [2]:
print(h5py.__version__)

0.4.1


In [3]:
! hsls -H -v /home/jreadey/ESIP/

jreadey@hdfgroup.org                 folder   2019-05-04 14:15:37 /home/jreadey/ESIP/
jreadey@hdfgroup.org   104.1K        domain   2019-06-06 20:12:39 /home/jreadey/ESIP/adcirc_01.nc
2 items


## Try reading with h5pyd (0.4.0 version)

In [4]:
f = h5py.File("/home/jreadey/ESIP/adcirc_01.nc", "r")

In [5]:
zeta = f['zeta']

In [6]:
zeta.chunks

(10, 141973)

In [7]:
zeta.shape

(720, 9228245)

Let's try reading one chunk of data

In [8]:
n = 141973

In [9]:
%time arr = f['zeta'][:320, 2*n:3*n]

CPU times: user 780 ms, sys: 1.2 s, total: 1.98 s
Wall time: 14.5 s


How many chunks are there?

In [10]:
f['zeta']

<HDF5 dataset "zeta": shape (720, 9228245), type "<f8">

In [11]:
nchunks = np.prod(zeta.shape) / np.prod(zeta.chunks)
print(nchunks)

4680.0


In [12]:
f['zeta']

<HDF5 dataset "zeta": shape (720, 9228245), type "<f8">

In [13]:
np.prod(zeta.chunks)*8/1e6

11.35784

## Now try reading with xarray (using h5netcdf + h5pyd)

In [14]:
import xarray as xr

In [23]:
url_hsds = 'http://hsdshdflab.hdfgroup.org/home/jreadey/ESIP/adcirc_01.nc'
url_hsds = "http://hsdshdflab.hdfgroup.org/home/jreadey/ESIP/adcirc_native.nc"

In [24]:
from dask.distributed import Client, progress
from dask_kubernetes import KubeCluster

In [25]:
cluster = KubeCluster()

In [26]:
cluster.scale(4);

In [27]:
cluster

In [28]:
cluster.adapt(maximum=4)

In [29]:
client = Client(cluster)

In [30]:
ds_hsds = xr.open_dataset(url_hsds, engine='h5netcdf', chunks={'time':10, 'node':141973})

In [31]:
ds_hsds['zeta']

<xarray.DataArray 'zeta' (time: 720, node: 9228245)>
dask.array<shape=(720, 9228245), dtype=float64, chunksize=(10, 141973)>
Coordinates:
  * time     (time) datetime64[ns] 2031-08-03T02:10:00 ... 2031-08-08T02:00:00
    x        (node) float64 dask.array<shape=(9228245,), chunksize=(141973,)>
    y        (node) float64 dask.array<shape=(9228245,), chunksize=(141973,)>
Dimensions without coordinates: node
Attributes:
    long_name:      water surface elevation above geoid
    standard_name:  sea_surface_height_above_geoid
    location:       node
    mesh:           adcirc_mesh
    units:          m

In [32]:
%%time
max_var = ds_hsds['zeta'].max(dim='time').compute()

CPU times: user 2min 45s, sys: 11.7 s, total: 2min 57s
Wall time: 13min 32s
